# Apache Drill in Action

Apache Drill is a Unix service that unifies access to data across a variety of data formats and sources. It can run on a single node or on multiple nodes in a clustered environment. It supports a variety of NoSQL databases and file systems, including HBase, MongoDB, HDFS, Amazon S3, Azure Blob Storage, and local text files like json or csv files. Drill provides the following user friendly features:
- Drill supports industry standard APIs such as ANSI SQL and JDBC
- Drill enables you to perform queries without predefining schemas
- Drill enables you to JOIN data in different formats from multiple datastores.

This Jupyter notebook shows how to join relational and nonrelational datasets with Apache Drill.

For more information about programming with Drill, see [the Apache Drill docs](https://drill.apache.org/docs/installing-the-driver-on-linux/).


In [1]:
# Import modules
import pyodbc
from pandas import *
import pandas as pd
import unicodedata
# Show datasources defined in ~/.odbc.ini
print(pyodbc.dataSources());

{'ODBC': '', 'drill64': '/opt/mapr/drillodbc/lib/64/libdrillodbc_sb64.so'}


In [2]:
# Initialize the connection
conn = pyodbc.connect("DSN=drill64;CHARSET=UTF8", autocommit=True)
conn.setdecoding(pyodbc.SQL_WMETADATA, encoding='utf-32le')
cursor = conn.cursor()

## SELECT from a MapR-DB table:

In [8]:
data = pandas.read_sql("SELECT _id, name, address, phone_number, latitude, longitude, zip, first_visit, churn_risk, sentiment FROM `dfs.default`.`./tmp/crm_data` limit 100", conn)
data.address[0:10]

0         4 8 5 7 8   S a n d y   M a p l e   E d g e 
1               6 4   P r e t t y   R o c k   D e l l 
2    3 2 2 7 9   G r a n d   C a s t l e   S t r e ...
3     7 1 0 3   H e a t h e r   A x e   E s t a t e s 
4    1 5 3 8 8   S u n n y   B e a v e r   A v e n ...
5    4 3 2 3 8   H i g h   S t o n e   P r o m e n ...
6    1 5 4 5   M e r r y   B r i d g e   O r c h a ...
7    1 3 9 8 4   T h u n d e r   T u r t l e   S u ...
8    3 5 9 0   T u r n i n g   B u t t e r n u t   ...
9       3 6 7 2   R o s e   A u t u m n   V a l l e y 
Name: address, dtype: object

## SELECT from a CSV file:

In [9]:
# setup the query and run it
s = "SELECT * FROM `dfs.tmp`.`./companylist.csv2` limit 3"
data = pandas.read_sql(s, conn)
data

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,fqn,filename,filepath,suffix
0,P I H ,1 3 4 7   P r o p e r t y   I n s u r a n c e ...,7 . 2 ,4 2 9 0 3 8 3 5 . 2 ,n / a ,2 0 1 4 ,F i n a n c e ,P r o p e r t y - C a s u a l t y   I n s u r ...,h t t p : / / w w w . n a s d a q . c o m / s ...,/ t m p / c o m p a n y l i s t . c s v 2 ,c o m p a n y l i s t . c s v 2 ,/ t m p ,c s v 2 
1,F L W S ,"1 - 8 0 0   F L O W E R S . C O M ,   I n c . ",1 0 . 2 ,6 6 8 8 0 4 3 9 1 . 6 ,n / a ,1 9 9 9 ,C o n s u m e r   S e r v i c e s ,O t h e r   S p e c i a l t y   S t o r e s ,h t t p : / / w w w . n a s d a q . c o m / s ...,/ t m p / c o m p a n y l i s t . c s v 2 ,c o m p a n y l i s t . c s v 2 ,/ t m p ,c s v 2 
2,F C C Y ,1 s t   C o n s t i t u t i o n   B a n c o r ...,1 8 . 9 5 ,1 5 1 4 8 2 3 0 1 . 5 5 ,n / a ,n / a ,F i n a n c e ,S a v i n g s   I n s t i t u t i o n s ,h t t p : / / w w w . n a s d a q . c o m / s ...,/ t m p / c o m p a n y l i s t . c s v 2 ,c o m p a n y l i s t . c s v 2 ,/ t m p ,c s v 2 


In [6]:
# Fetch and display filtered output
cursor.execute(s)
row = cursor.fetchone() 
print row[0], row[1]


P I H  1 3 4 7   P r o p e r t y   I n s u r a n c e   H o l d i n g s ,   I n c . 


## SELECT from a relational database (MySQL):

In [12]:
# Here's how to select from MySQL
s = "select * from ianmysql.cars.`car`"
data = pandas.read_sql(s, conn)
data

,customerid,name
0,0,a u d i 
1,1,b m x 
2,2,t o y o t a 
3,3,f o r d 


## SELECT from a JSON file:

In [13]:
s = "SELECT id,name FROM `dfs.tmp`.`./names.json` limit 3"
data = pandas.read_sql(s, conn)
data

,id,name
0,1,E r i c a   R o m a g u e r a 
1,2,C a l e i g h   J e r d e 
2,3,L u c a s   S c h u l t z 


In [14]:
s = "SELECT id,address FROM `dfs.tmp`.`./addressunitedstates.json` limit 3"
data = pandas.read_sql(s, conn)
data

,id,address
0,1,"7 2 8   D o o l e y   B r a n c h ,   B e c k ..."
1,2,"6 2 2   D i x i e   P a t h ,   S o u t h   T ..."
2,3,"2 0 4 6 4   L y n c h   H a v e n ,   E m a r ..."


## JOIN fields across two JSON files:

In [33]:
# Here's how to do a JOIN
s = "SELECT tbl1.name, tbl2.address FROM `dfs.tmp`.`./names.json` as tbl1 JOIN `dfs.tmp`.`./addressunitedstates.json` as tbl2 ON tbl1.id=tbl2.id"
data = pandas.read_sql(s, conn)
data

,n a ,a d d r
0,E r i c a   R o m a g u e r a ,"7 2 8   D o o l e y   B r a n c h ,   B e c k ..."
1,C a l e i g h   J e r d e ,"6 2 2   D i x i e   P a t h ,   S o u t h   T ..."
2,L u c a s   S c h u l t z ,"2 0 4 6 4   L y n c h   H a v e n ,   E m a r ..."


## JOIN fields across a JSON file and a relational database:

In [16]:
# JOIN two fields
s = "SELECT tbl1.name, tbl2.name FROM `dfs.tmp`.`./names.json` as tbl1 JOIN ianmysql.cars.`car` as tbl2 ON tbl1.id=tbl2.customerid"
data = pandas.read_sql(s, conn)
data

,name,name0
0,E r i c a   R o m a g u e r a ,b m x 
1,C a l e i g h   J e r d e ,t o y o t a 
2,L u c a s   S c h u l t z ,f o r d 


In [17]:
# JOIN three fields
s = "SELECT tbl1.name, tbl2.address, tbl3.name as car FROM `dfs.tmp`.`./names.json` as tbl1 JOIN `dfs.tmp`.`./addressunitedstates.json` as tbl2 ON tbl1.id=tbl2.id JOIN ianmysql.cars.`car` as tbl3 ON tbl1.id=tbl3.customerid"
data = pandas.read_sql(s, conn)
data

,name,address,car
0,E r i c a   R o m a g u e r a ,"7 2 8   D o o l e y   B r a n c h ,   B e c k ...",b m x 
1,C a l e i g h   J e r d e ,"6 2 2   D i x i e   P a t h ,   S o u t h   T ...",t o y o t a 
2,L u c a s   S c h u l t z ,"2 0 4 6 4   L y n c h   H a v e n ,   E m a r ...",f o r d 


## SELECT data from our Customer 360 CRM dataset in MapR-DB:

In [21]:
sql="SELECT * FROM `dfs.default`.`./tmp/crm_data`"
df=pd.read_sql(sql, conn)

In [22]:
df.head(5)

,_id,address,birth_date,churn_risk,email,first_visit,gender,id,latitude,longitude,name,persona,phone_number,sentiment,ssn,state,zip
0,0 0 0 0 2 2 e 6 - 1 6 3 d - 4 b 2 d - 8 5 e 9 ...,4 8 5 7 8   S a n d y   M a p l e   E d g e ,0 4 / 2 2 / 2 0 0 1 ,6.0,C a r l T a y l o r @ e x a m p l e . c o m ,0 7 / 1 5 / 2 0 1 2 ,F E M A L E ,0 0 0 0 2 2 e 6 - 1 6 3 d - 4 b 2 d - 8 5 e 9 ...,3 1 . 9 5 ,- 9 2 . 7 7 ,E v a   P e t e r s o n ,3.0,9 7 2 - 3 1 9 - 5 9 4 9 ,P O S I T I V E ,4 4 8 - 1 5 - 9 2 4 0 ,w i ,7 1 4 1 0 
1,0 0 0 0 3 d 0 7 - e e 7 3 - 4 f a 8 - 9 e 3 e ...,6 4   P r e t t y   R o c k   D e l l ,0 8 / 0 8 / 1 9 7 4 ,2.0,R o s l y n S o l o m o n @ e x a m p l e . c ...,1 0 / 2 6 / 2 0 0 3 ,F E M A L E ,0 0 0 0 3 d 0 7 - e e 7 3 - 4 f a 8 - 9 e 3 e ...,3 8 . 4 5 ,- 8 2 . 6 4 ,R i c h a r d   E s c a l a n t e ,2.0,2 7 6 - 4 6 4 - 8 0 9 0 ,N E U T R A L ,5 3 0 - 7 8 - 7 3 0 6 ,g a ,4 1 1 0 1 
2,0 0 0 0 c d d 9 - 0 3 5 e - 4 9 3 a - b 1 c f ...,3 2 2 7 9   G r a n d   C a s t l e   S t r e ...,0 7 / 1 5 / 1 9 4 9 ,20.0,S a n d r a D a v i s @ e x a m p l e . c o m ,1 2 / 0 5 / 2 0 0 2 ,M A L E ,0 0 0 0 c d d 9 - 0 3 5 e - 4 9 3 a - b 1 c f ...,4 1 . 3 3 ,- 9 0 . 6 7 ,J o a n   P a y n e ,1.0,8 0 9 - 3 5 8 - 1 7 4 5 ,P O S I T I V E ,0 3 4 - 9 9 - 8 1 2 0 ,m i ,6 1 2 7 9 
3,0 0 0 0 f a d 6 - 8 1 d 1 - 4 5 b d - 8 6 6 d ...,7 1 0 3   H e a t h e r   A x e   E s t a t e s ,1 2 / 2 1 / 1 9 5 5 ,4.0,W a y n e H u l s @ e x a m p l e . c o m ,0 3 / 1 9 / 2 0 1 0 ,M A L E ,0 0 0 0 f a d 6 - 8 1 d 1 - 4 5 b d - 8 6 6 d ...,3 8 . 7 3 ,- 9 9 . 1 2 ,R a c h e l   W o r r e l l ,3.0,7 8 8 - 6 6 6 - 9 4 7 1 ,N E G A T I V E ,4 6 7 - 1 7 - 5 3 0 4 ,m a ,6 7 6 6 0 
4,0 0 0 2 a 2 9 0 - 2 1 e e - 4 3 6 7 - 9 b e 7 ...,1 5 3 8 8   S u n n y   B e a v e r   A v e n ...,0 2 / 1 8 / 1 9 4 5 ,5.0,I s i a h I r v i n g @ e x a m p l e . c o m ,1 0 / 1 4 / 2 0 0 3 ,M A L E ,0 0 0 2 a 2 9 0 - 2 1 e e - 4 3 6 7 - 9 b e 7 ...,3 9 . 5 7 ,- 7 6 . 8 0 ,N a t h a n   P o r t e r ,2.0,7 9 9 - 3 0 5 - 3 6 4 1 ,N E U T R A L ,5 3 0 - 6 5 - 8 5 6 5 ,p a ,2 1 1 5 5 


## Filter using a WHERE clause:

In [25]:
sql="SELECT * FROM `dfs.tmp`.`./crm_data` where ssn='448-15-9240'"
df=pd.read_sql(sql, conn)
df

,_id,address,birth_date,churn_risk,email,first_visit,gender,id,latitude,longitude,name,persona,phone_number,sentiment,ssn,state,zip
0,0 0 0 0 2 2 e 6 - 1 6 3 d - 4 b 2 d - 8 5 e 9 ...,4 8 5 7 8   S a n d y   M a p l e   E d g e ,0 4 / 2 2 / 2 0 0 1 ,6.0,C a r l T a y l o r @ e x a m p l e . c o m ,0 7 / 1 5 / 2 0 1 2 ,F E M A L E ,0 0 0 0 2 2 e 6 - 1 6 3 d - 4 b 2 d - 8 5 e 9 ...,3 1 . 9 5 ,- 9 2 . 7 7 ,E v a   P e t e r s o n ,3.0,9 7 2 - 3 1 9 - 5 9 4 9 ,P O S I T I V E ,4 4 8 - 1 5 - 9 2 4 0 ,w i ,7 1 4 1 0 


In [27]:
sql="SELECT * FROM `dfs.tmp`.`./crm_data` where name='Erika Gallardo'"
df=pd.read_sql(sql, conn)
df

,_id,address,birth_date,churn_risk,email,first_visit,gender,id,latitude,longitude,name,persona,phone_number,sentiment,ssn,state,zip
0,0 0 0 3 c 1 2 c - e e 2 e - 4 a b 8 - a 1 d c ...,1 5 4 5   M e r r y   B r i d g e   O r c h a ...,1 0 / 2 1 / 1 9 8 1 ,29.0,K e v i n P a l m e r @ e x a m p l e . c o m ,0 1 / 2 5 / 2 0 1 2 ,F E M A L E ,0 0 0 3 c 1 2 c - e e 2 e - 4 a b 8 - a 1 d c ...,4 4 . 9 4 ,- 9 3 . 1 0 ,E r i k a   G a l l a r d o ,3.0,8 1 7 - 9 3 8 - 5 6 4 8 ,P O S I T I V E ,5 2 9 - 4 2 - 0 5 5 3 ,i n ,5 5 1 1 7 


In [29]:
sql="SELECT * FROM `dfs.tmp`.`./crm_data` where name like '%bright%'"
df=pd.read_sql(sql, conn)
df.dropna().head(5)

,_id,address,birth_date,churn_risk,email,first_visit,gender,id,latitude,longitude,name,persona,phone_number,sentiment,ssn,state,zip
0,1 6 0 b 1 8 6 a - a e 6 a - 4 1 6 3 - 9 8 a 8 ...,5 5 8 3   G o l d e n   L a k e   C a n y o n ,1 2 / 3 1 / 1 9 9 6 ,4.0,V i c t o r S m a l l @ e x a m p l e . c o m ,0 8 / 2 6 / 1 9 9 8 ,M A L E ,1 6 0 b 1 8 6 a - a e 6 a - 4 1 6 3 - 9 8 a 8 ...,4 0 . 3 1 ,- 7 5 . 0 6 ,J o h n n y   A l b r i g h t ,2.0,6 6 3 - 6 5 4 - 6 0 9 0 ,N E U T R A L ,5 7 9 - 6 9 - 1 1 1 5 ,i n ,1 8 9 1 2 
1,1 6 5 7 3 a c 6 - 8 7 f 3 - 4 f a d - 8 1 d e ...,4 3 2 8 0   H i d d e n   S w a n   C u t ,0 7 / 1 7 / 1 9 9 4 ,16.0,N e l l S e a m o n @ e x a m p l e . c o m ,0 5 / 2 1 / 1 9 9 5 ,M A L E ,1 6 5 7 3 a c 6 - 8 7 f 3 - 4 f a d - 8 1 d e ...,3 5 . 1 0 ,- 9 0 . 0 0 ,F r a n k   H a m b r i g h t ,3.0,1 9 3 - 3 9 3 - 5 2 9 3 ,P O S I T I V E ,2 6 7 - 7 1 - 8 3 4 8 ,i l ,3 8 1 8 8 
2,2 1 7 4 b 3 c 1 - 6 8 3 1 - 4 5 2 5 - a e 6 f ...,3 0 1 1   C i n d e r   P r a i r i e   G l e n ,0 5 / 3 0 / 1 9 3 6 ,16.0,M a r t h a W i l e y @ e x a m p l e . c o m ,0 1 / 2 4 / 2 0 1 3 ,F E M A L E ,2 1 7 4 b 3 c 1 - 6 8 3 1 - 4 5 2 5 - a e 6 f ...,3 4 . 1 5 ,- 1 1 8 . 5 4 ,R i c k y   A l b r i g h t ,2.0,6 1 9 - 9 2 0 - 9 4 5 5 ,N E U T R A L ,3 1 7 - 4 4 - 2 6 4 4 ,n y ,9 1 3 5 6 
3,2 2 9 0 3 2 8 2 - a 3 b 7 - 4 3 9 e - 8 6 9 1 ...,6 9 7 5   T e n d e r   A x e   P a c e ,0 5 / 3 1 / 1 9 8 1 ,6.0,J a c k i e B u r k e @ e x a m p l e . c o m ,0 8 / 1 5 / 2 0 0 3 ,F E M A L E ,2 2 9 0 3 2 8 2 - a 3 b 7 - 4 3 9 e - 8 6 9 1 ...,4 2 . 3 5 ,- 7 1 . 0 6 ,M i c h e l l e   A l b r i g h t ,2.0,4 5 7 - 6 5 1 - 5 4 5 5 ,P O S I T I V E ,5 0 0 - 2 8 - 5 8 0 0 ,p a ,0 2 2 9 7 
4,2 a a 7 a 6 2 7 - a 6 c d - 4 1 8 5 - a 0 1 4 ...,6 4 3 7 0   H a p p y   H o r s e   N i c h e ,0 5 / 3 1 / 1 9 3 7 ,10.0,K a t h l e e n L a m @ e x a m p l e . c o m ,0 9 / 0 9 / 2 0 0 2 ,F E M A L E ,2 a a 7 a 6 2 7 - a 6 c d - 4 1 8 5 - a 0 1 4 ...,3 5 . 8 2 ,- 7 8 . 6 5 ,J u s t i n   F u l b r i g h t ,2.0,5 5 4 - 2 6 5 - 6 1 8 5 ,N E U T R A L ,5 2 9 - 5 5 - 2 0 8 4 ,w a ,2 7 6 3 5 


## Instead of using a WHERE clause you can also filter using standard Python dataframe functions:

In [31]:
conn = pyodbc.connect("DSN=drill64", autocommit=True)
conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-32le', to=str)
conn.setdecoding(pyodbc.SQL_WMETADATA, encoding='utf-32le', to=str)
cursor = conn.cursor()
sql = "SELECT * FROM `dfs.tmp`.`./salary_data.csv2`"
df=pd.read_sql(sql, conn)
df[(df['name'].str.contains("ea"))]

,name,salary,years_experience,fqn,filename,filepath,suffix
0,Keagan Klocko,92239.0868697,4,/tmp/salary_data.csv2,salary_data.csv2,/tmp,csv2
3,Shana Beahan,59978.9111169,3,/tmp/salary_data.csv2,salary_data.csv2,/tmp,csv2
16,Mireya Heathcote,48815.1639106,5,/tmp/salary_data.csv2,salary_data.csv2,/tmp,csv2
84,Deangelo Swaniawski,55705.7234414,3,/tmp/salary_data.csv2,salary_data.csv2,/tmp,csv2
103,Jeanne Schaefer,76916.9020765,7,/tmp/salary_data.csv2,salary_data.csv2,/tmp,csv2
163,Mayra Gleason,72661.5777432,4,/tmp/salary_data.csv2,salary_data.csv2,/tmp,csv2
232,Breana Gottlieb,41255.6678519,0,/tmp/salary_data.csv2,salary_data.csv2,/tmp,csv2
